In [3]:
input_txt = 'Bonjour, mon chien est mignon.'
target_lang = 'eng'

# LID (language identification)
Using https://huggingface.co/facebook/fasttext-language-identification  
this model is CPU only but is pretty fast

In [4]:
import fasttext
from huggingface_hub import hf_hub_download

# load model
model_path = hf_hub_download(repo_id="facebook/fasttext-language-identification", filename="model.bin")
model = fasttext.load_model(model_path)

# predict, outputs:((prediction, probablity))
lid_prediction = model.predict(input_txt)

# extract lang code from prediction
lid = lid_prediction[0][0]
detected_source_lang = lid[lid.rfind("__")+2:lid.rfind("_")]

In [5]:
detected_source_lang

'fra'

# Txt translation
using SeamlessM4T Large (https://huggingface.co/facebook/hf-seamless-m4t-large) "seamless" but with no LID!! raah!    
Max word count input: 4096. May need to adjust smaller for good resutls; plz test  
Can use CPU or GPU; below uses GPU

In [6]:
import torch
from transformers import AutoProcessor, SeamlessM4TForTextToText

# cuda or cpu
device = 'cuda'

# load model & tokenizer
tokenizer = AutoProcessor.from_pretrained("facebook/hf-seamless-m4t-large", device_map = device, use_fast = False)
model = SeamlessM4TForTextToText.from_pretrained("facebook/hf-seamless-m4t-large", device_map = device)

# txt to txt translation (source language required)
input_tokens = tokenizer(text = input_txt, src_lang=detected_source_lang, return_tensors="pt").to(torch.device(device))
output_tokens = model.generate(**input_tokens, tgt_lang=target_lang)
translated_text = tokenizer.decode(output_tokens[0].tolist(), skip_special_tokens=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
translated_text

'Hi, my dog is cute.'